In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
#importer les données et les afficher
t = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
t.head()

#Interprétation des paramètres : 
Pregnancies = nombre de grossesse
Glucose = taux de glucose
BloodPressure = Pression dans le sang 
SkinThickness = Epaisseur de la peau
Insulin = insulin 
BMI = IMC / 
DiabetesPedigreeFunction = l'hérédité / 
Age = age 
Outcome = Résultat (0= pas de diabète, 1 = diabète)

In [ ]:
display(t.info(),t.head())

In [ ]:
#Comptage par colonne
t.count()

In [ ]:
# Compléter les données manquantes

diabetes = t.fillna(value = {'Glucose':t.BloodPressure.mean()})

In [ ]:
 plt.hist(t.Glucose,bins=100)
    

In [ ]:
diabetes.count()

In [ ]:
diabetes1 = diabetes.fillna(value = {'SkinThickness':t.SkinThickness.mean()})

In [ ]:
plt.hist(diabetes1.SkinThickness, bins=100)

In [ ]:
diabetes1.count()

In [ ]:
# BloodPressure

diabetes2 = diabetes1.fillna(value = {'BloodPressure':t.BloodPressure.mean()})

In [ ]:
plt.hist(diabetes2.BloodPressure, bins=100)

In [ ]:
diabetes2.count()

In [ ]:
diabetes3 = diabetes2.fillna(value = {'Insulin':t.Insulin.mean()})

In [ ]:
plt.hist(diabetes3.Insulin, bins=100)

In [ ]:
diabetes3.count()

In [ ]:
diabetes4 = diabetes3.fillna(value = {'BMI':t.BMI.mean()})

In [ ]:
plt.hist(diabetes4.BMI, bins=100)

In [ ]:
diabetes4.count()

In [ ]:
# distributionsdéséquilibrées

sns.distplot(diabetes4.Insulin, color='blue')

In [ ]:
# Amélioration de l'équilibre avec une transformation log

diabetes4['log_Insulin'] = np.log(diabetes4.Insulin+1)

In [ ]:
sns.kdeplot(diabetes4.log_Insulin, color='blue')

In [ ]:
diabetes4[['Age','log_Insulin']].describe()

In [ ]:
sns.kdeplot(diabetes4.log_Insulin, color='blue')
sns.kdeplot(diabetes4.Age, color='red')

In [ ]:
from sklearn import preprocessing

In [ ]:
#Normaliser les valeurs min et à max (valeurs ramenées entre 0 et 1)

minmax = preprocessing.MinMaxScaler(feature_range=(0, 1))
diabetes4[['Age', 'log_Insulin']] = minmax.fit_transform(diabetes4[['Age', 'log_Insulin']])

In [ ]:
sns.distplot(diabetes4.log_Insulin, color='blue')
sns.distplot(diabetes4.Age, color='red')

In [ ]:
#StandardScaler pour ramener la moyenne à 0 et l'écart type à 1

scaler = preprocessing.StandardScaler()
diabetes4[['Age', 'log_Insulin']] = scaler.fit_transform(diabetes4[['Age', 'log_Insulin']])

In [ ]:
sns.kdeplot(diabetes4.log_Insulin, color='blue')
sns.kdeplot(diabetes4.Age, color='red')

In [ ]:
X = diabetes4.drop(['Outcome'], axis=1)
y = diabetes4.Outcome

In [ ]:
#importer train test split function de sklearn library

from sklearn.model_selection import train_test_split

In [ ]:
# On sépare le dataset en deux parties (80% pour l'apprentissage et 30% pour le test) 

X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
#Régression logistique

from sklearn.linear_model import LogisticRegression


In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

In [ ]:
#On importe le classificateur de la librairie sklearn

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

In [ ]:
# Évaluer la matrice de confusion

print(confusion_matrix(y_test,y_lr))

In [ ]:
# Évaluer la qualité de la précision avec sklearn library

print(accuracy_score(y_test,y_lr))

In [ ]:
# Évaluer le rappart de classification 

print(classification_report(y_test, y_lr))

In [ ]:
#Tableau de couples de probabilités

probas = lr.predict_proba(X_test)

In [ ]:
print(probas)

In [ ]:
#Dataframe

dfprobas = pd.DataFrame(probas,columns=['proba_0','proba_1'])
dfprobas['y'] = np.array(y_test)

In [ ]:
dfprobas

In [ ]:
# Graphique de distribution des probabilités de prédiction

plt.figure(figsize=(10,10))
sns.distplot(1-dfprobas.proba_0[dfprobas.y==0], bins=50)
sns.distplot(dfprobas.proba_1[dfprobas.y==1], bins=50)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

In [ ]:
plt.figure(figsize=(12,12))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')        
plt.plot([0,0,1],[0,1,1],'g:')     
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

In [ ]:
#Random forest

from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

In [ ]:
rf1 = ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=10, max_features=3)
rf1.fit(X_train, y_train)
y_rf1 = rf.predict(X_test)
print(classification_report(y_test, y_rf1))

In [ ]:
# La validation_curve 

from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'n_estimators', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('n_estimators')
plt.ylabel('score');

In [ ]:
from sklearn import model_selection

In [ ]:
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

In [ ]:
rf_gs.fit(X_train, y_train)

In [ ]:
print(rf_gs.best_params_)

In [ ]:
rf2 = rf_gs.best_estimator_

In [ ]:
y_rf2 = rf2.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf2))

In [ ]:
#Importance des caractéristiques

importances = rf2.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')

In [ ]:
#XGBOOST

!pip install xgboost

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
cm = confusion_matrix(y_test, y_xgb)
print(cm)
print(classification_report(y_test, y_xgb))

In [ ]:
#modification de dataframe
def replace_0(df,col) :
    df1 = df.copy()
    n = df.shape[0]
    m = df[col].mean()
    s = df[col].std()
    for i in range(n) :
        if df.loc[i,col]==0 :
            df1.loc[i,col] = np.random.normal(m,s)
    return df1

In [ ]:
df = replace_0(df,"Glucose")